https://www.makeuseof.com/python-hand-tracking-opencv/

In [2]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [7]:
# Read the previously saved calibration data
filename = os.path.relpath('../../CameraCalibration/Calibration');

data = np.load(filename +"_camera.npz")
mtx = data['mtx']
dist = data['dist']

In [87]:

def add_hologram_effect(image, hologram,corners,alpha=0.5):
    """Adds a hologram effect to an image."""

    hologram=map_image_to_rectangle(hologram,corners,image.shape) 

    # Create a shimmering effect for the hologram
    shimmer = np.random.rand(hologram.shape[0], hologram.shape[1], 3) * 255
    shimmer = shimmer.astype(np.uint8)

    # Overlay the hologram with shimmering effect onto the image
    result = cv2.addWeighted(image, 1 - alpha, hologram, alpha, 0)
    result = cv2.addWeighted(result, 1, shimmer, 0.1, 0)

    return result

def map_image_to_rectangle(image, corners,newsize):
    """Maps the given image to the specified rectangle corners.

    Args:
        image (numpy.ndarray): The input image.
        corners (numpy.ndarray): A 4x2 array of corner points in the following order:
            top-left, top-right, bottom-right, bottom-left.

    Returns:
        numpy.ndarray: The warped image.
    """

    # Define the target rectangle corners
    width = image.shape[0]
    height = image.shape[1]
    dst_corners = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

    # Compute the perspective transformation matrix
    M = cv2.getPerspectiveTransform(dst_corners,corners.astype(np.float32))

    # Warp the image
    warped_image = cv2.warpPerspective(image, M, (newsize[0],newsize[1]))

    return warped_image


In [96]:
# Load the image and the hologram image
image = cv2.imread(os.path.relpath('../../Data/ROC2.jpeg'))
hologram = cv2.imread(os.path.relpath('../../Data/PittPanthersLogo.png'), cv2.IMREAD_UNCHANGED)
hologram=hologram[:,:,:-1]

image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
hologram = cv2.cvtColor(hologram,cv2.COLOR_BGR2RGB)

corners=np.array(((1500,1500),(1800,1500),(1800,1800),(1500,1800)))

# Add the hologram effect
result = add_hologram_effect(image, hologram,corners,alpha=.5)


In [98]:
import cv2
import mediapipe as mp
import imutils

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Processing the input image
def process_image(img):
    # Converting the input to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(gray_image)

    # Returning the detected hands to calling function
    return results

# Drawing landmark connections
def draw_hand_connections(img, results):
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape

                # Finding the coordinates of each landmark
                cx, cy = int(lm.x * w), int(lm.y * h)

                # Printing each landmark ID and coordinates
                # on the terminal
                print(id, cx, cy)

                # Creating a circle around each landmark
                cv2.circle(img, (cx, cy), 10, (0, 255, 0),
                           cv2.FILLED)
                # Drawing the landmark connections
                mpDraw.draw_landmarks(img, handLms,
                                      mpHands.HAND_CONNECTIONS)

        return img

ModuleNotFoundError: No module named 'mediapipe'

In [ ]:

cap = cv2.VideoCapture(0)

while True:
    # Taking the input
    success, image = cap.read()
    image = imutils.resize(image, width=500, height=500)
    results = process_image(image)
    draw_hand_connections(image, results)

    # Displaying the output
    cv2.imshow("Hand tracker", image)

    # Program terminates when q key is pressed
    if cv2.waitKey(1) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()